In [20]:
import os
import pathlib as Path
import json
import pandas as pd
from zeep import Client
import hashlib

PRUEBAS=True
FECHA_INICIO = '2025-03-21'
FECHA_FIN = ''
if PRUEBAS:
    BASE_URL = 'http://pruebas1.techra.com.mx/WebService/'
    USUARIO = 'cliente_prueba'
    PASSWD= 'c27c2a5cdbcd7542dd00bb6819f8c1ce'
else:
    BASE_URL = 'https://ara.techra.com.mx/WebService/'
    USUARIO = 'javier_a'
    PASSWD= '129c9b723d04dee6c903db36aab49fbf'

def inicia_sesion(usuario, password, IdSessionAnterior=any):
    url = f'{BASE_URL}AutenticaUsuario.php?wsdl'
    cliente_inicia_sesion = Client(url)
    res = cliente_inicia_sesion.service.autenticaUsuario(
        usuario,
        password,
        ''
    )
    return res

def get_usuarios(id_session):
    cliente_usuarios = Client(f'{BASE_URL}ListaUsuario.php?wsdl')
    res = cliente_usuarios.service.getUsuario(id_session, '')
    return res

def get_lista_usuarios(id_session):
    cliente_lista_usuarios = Client(f'{BASE_URL}ListaUsuario.php')

def get_localizacion(session, usuario):
    url = f'{BASE_URL}LocalizaUsuario.php?wsdl'
    cliente_localizacion = Client(url)
    res = cliente_localizacion.service.getUbicaciones(usuario,session)
    return res

def get_tickets_por_fecha(session, idusuario, ticket, fechainicio, fechafin):
    url = f'{BASE_URL}ConsultaTicket.php?wsdl'
    cliente_tickets = Client(url)
    res = cliente_tickets.service.getTickets(
        session,
        idusuario, #USUARIO
        '', #status
        fechainicio, #fecha
        fechafin, #fechafin
        '', #latitud
        '', #longitud
        '', #pagina
        '', #tampagina
        ticket, #ticket
        '', #serie
        0, #cerrados
        0, #cancelados
        0, #morosos
        '', #tipo
        '', #cliente
        ''  #todos
    )
    return res

def get_tickets(session, usuario, ticket):
    url = f'{BASE_URL}ConsultaTicket.php?wsdl'
    cliente_tickets = Client(url)
    res = cliente_tickets.service.getTickets(
        session,
        usuario,
        '', #status
        '', #fecha
        '', #fechafin
        '', #latitud
        '', #longitud
        '', #pagina
        '', #tampagina
        ticket, #ticket
        '', #serie
        0, #cerrados
        0, #cancelados
        0, #morosos
        '', #tipo
        '', #cliente
        ''  #todos
    )
    return res

def get_notas_por_ticket(ticket, session):
    url = f'{BASE_URL}ConsultaNota.php?wsdl'
    cliente_notas = Client(url)
    res = cliente_notas.service.getNotas(
        ticket,
        '',
        '',
        '',
        session
    )
    return res

def get_refacciones_por_equipo(equipo,session):
    url = f'{BASE_URL}ObtenerParteComponente.php?wsdl'
    cliente_refacciones = Client(url)
    res = cliente_refacciones.service.obtenerComponentePorEquipo(
        equipo,
        session
    )
    return res

def lista_json_to_excel(lista_json, nombre_archivo):
    if len(lista_json) > 0:
        keys = lista_json[0].keys()
        columns = [key for key in keys]
        data = []
        for json in lista_json:
            data.append([json[key] for key in json.keys()])
            # data.append([values[0], values[1]])
        df = pd.DataFrame(data, columns=columns)
        df.to_excel(f'{nombre_archivo}.xlsx', index=False)
        return f'{nombre_archivo}.xlsx'
    else:
        return None

In [11]:
PRUEBAS=True
FECHA_INICIO = '2025-01-01'
if PRUEBAS:
    BASE_URL = 'http://pruebas1.techra.com.mx/WebService/'
    USUARIO = 'cliente_prueba'
    PASSWD= 'c27c2a5cdbcd7542dd00bb6819f8c1ce'
else:
    BASE_URL = 'https://ara.techra.com.mx/WebService/'
    USUARIO = 'javier_a'
    PASSWD= '129c9b723d04dee6c903db36aab49fbf'

In [21]:
res = inicia_sesion(USUARIO, PASSWD)

In [23]:
json.loads(res)

[{'FechaCreacion': '2025-04-02 09:37:50',
  'IdSession': 'UQ7]+YKmLXCnvK*',
  'DuracionMinutos': 1200,
  'IdUsuario': '584',
  'PermisoEP': 3,
  'IdEmpresa': '1',
  'IdPuesto': '106',
  'MinutosMonitoreoAnterior': '60',
  'MinutosMonitoreoPosterior': '30'}]

In [14]:
res = inicia_sesion(USUARIO, PASSWD)
session = res['IdSession']
id_usuario = res['IdUsuario']

In [15]:
resultado_tickets = get_tickets_por_fecha(session,'', '', '2025-01-01', '')
tickets = json.loads(resultado_tickets)

In [17]:
tickets[0]

{'Ticket': '533000',
 'Titulo': 'Repoirte de pruebas para mostrar asignacion automatica',
 'FechaCreacion': '2025-03-19',
 'Prioridad': None,
 'Zona': None,
 'Tecnico': 'miguellira',
 'AreaAtencion': 'Soporte ALT',
 'Serie': 'VD27409367',
 'ClaveCliente': '10182',
 'Cliente': 'DESC. CORPORATIVO, S.A. DE C.V.',
 'Localidad': 'GPM EXPENDIO SAN ANTONIO XLUCH (YUCATAN)',
 'Direccion': 'CALLE 86 (ENTRE 186 Y 185), No ext: 648 No. Int: 0, Col: SAN ANTONIO XLUCH, Del: MERIDA, Yucatán, México, C.P.: 97100',
 'Contacto1Nombre': 'KERMTIH MUÑOZ',
 'Contacto1Telefono': '9993002200 ext. 1338',
 'Contacto1Email': 'kermith.munoz@kuo.com.mx',
 'Contacto2Nombre': 'KERMTIH MUÑOZ',
 'Contacto2Telefono': '9993002200 ext. 1338',
 'Contacto2Email': 'kermith.munoz@kuo.com.mx',
 'Latitud': None,
 'Longitud': None,
 'Distancia': '0',
 'FechaNota': '2025-03-19 12:28:49',
 'IdTipoPrioridad': None,
 'TipoPrioridad': 'Baja',
 'IdColor': None,
 'Color': None,
 'MinutosAlertamiento': None,
 'IdPuestoEscalamiento': N

In [364]:
num_ticket = tickets[-2]['Ticket']
num_ticket

'532963'

In [365]:

resultado_notas = get_notas_por_ticket(num_ticket, session)
notas = json.loads(resultado_notas)

In [366]:
notas

[{'IdTicket': None,
  'Titulo': None,
  'Comentario': None,
  'NombreFoto': None,
  'Foto': None,
  'NombreUsuario': None,
  'FechaHora': None,
  'IdEstatus': None,
  'Estatus': None}]

In [18]:
usuarios = get_usuarios(session)
usuarios = json.loads(usuarios)
archivo_usuarios = lista_json_to_excel(usuarios, 'usuarios')

In [19]:
usuarios[0]

{'IdUsuario': '205', 'Nombre': 'Abigail Hernandez Ibarra'}

In [ ]:

resultado_tickets = get_tickets_por_fecha(session,'', FECHA_INICIO, FECHA_FIN)
tickets = json.loads(resultado_tickets)
archivo_tickets = lista_json_to_excel(tickets, 'tickets')
numeros_tickets = [t['Ticket'] for t in tickets[:-1]]
numeros_tickets = list(set(numeros_tickets))
lista_notas = []
for num_ticket in numeros_tickets:
    resultado_notas = get_notas_por_ticket(num_ticket, session)
    lista_notas += json.loads(resultado_notas)
archivo_notas = lista_json_to_excel(lista_notas, 'notas')

In [ ]:
numeros_serie = [t['Serie'] for t in tickets[:-1]]
numeros_serie_aux = []
for i, ns in enumerate(numeros_serie):
    if ns:
        serie_split = ns.split(',')
        if len(serie_split) > 1:
            for _ns in serie_split:
                numeros_serie_aux.append(_ns.strip())
        else:
            numeros_serie_aux .append(ns)
numeros_serie_aux = list(set(numeros_serie_aux))
sub_numeros_serie = numeros_serie_aux[10:50]

In [ ]:
lista_refacciones = []
for num_s in sub_numeros_serie:
    resultado_refacciones = get_refacciones_por_equipo(num_s, session)
    lista_aux = json.loads(resultado_refacciones)
    print(lista_aux)
    lista_refacciones += lista_aux
lista_refacciones = json.loads(resultado_refacciones)
archivo_refacciones = lista_json_to_excel(lista_refacciones, 'refacciones')


In [ ]:
resultado_refacciones = get_refacciones_por_equipo('042DD503', session)

In [ ]:
lista_notas = []
for num_ticket in numeros_tickets:
    resultado_notas = get_notas_por_ticket(num_ticket, session)
    lista_notas += json.loads(resultado_notas)

In [ ]:

len(lista_notas)

In [ ]:
notas

In [ ]:
resultado_tickets2 = get_tickets(session, '', '')
resultado_tickets2

In [ ]:
resultado_tickets = get_tickets(session, 584)


In [ ]:
tickets = json.loads(resultado_tickets)
print(len(tickets))


In [ ]:
tickets[0]

In [ ]:
resultado_ubicaciones = get_localizacion(session, '755')

In [ ]:
resultado_ubicaciones

In [ ]:
usuario = 'javier_a'
password = b'Arrieta2022'
password_md5 = hashlib.md5(password)
password_md5 = password_md5.hexdigest()
print(password_md5)


In [ ]:
password_md5

In [ ]:
print (session)

In [ ]:

usuarios = json.loads(usuarios)

In [ ]:
usuarios[0].keys()